In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None) #show all columns

import warnings
warnings.simplefilter('ignore') #to avoid warnings

## Exploring the Data

In [2]:
sae=pd.read_csv('../data/sae_dataset.csv')
sae.head()

,VehicleMarketCode,VehicleNationalCode,VehicleSegmentationGlobal,VehicleModelIntlName,Bodytype_Group,Fuel_Group,Transmission_Group,Drivetype_Group,VehicleKW,VehicleDoors,VehicleWheelBase,VehicleSeats,VehicleGrossNewPrice,VehicleProductionStart,VehicleProductionEnd,VehicleMakeIntlName,SAEReporteddate,StockDays,PostCode,registration_date,Mileage,Price,oricpric,PriceChangeCount
0,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,73,4.0,2716.0,2,19321.0,7/1/2004,12/1/2011,OPEL / VAUXHALL,2/3/2020,12,28983,7/1/2008,238000,2990,2990.0,NaN
1,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,73,4.0,2716.0,2,19321.0,7/1/2004,12/1/2011,OPEL / VAUXHALL,2/3/2020,86,8940,6/15/2010,90000,4499,4499.0,NaN
2,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,73,4.0,2716.0,2,19321.0,7/1/2004,12/1/2011,OPEL / VAUXHALL,2/3/2020,279,8840,7/1/2005,210000,2490,2700.0,1.0
3,ES,30020,E,607,Sedan,Diesel,Automated,2WD,150,4.0,2800.0,5,54250.0,12/1/2004,8/1/2012,PEUGEOT,10/12/2022,55,27002,10/15/2009,259000,6500,6500.0,NaN
4,ES,30020,E,607,Sedan,Diesel,Automated,2WD,150,4.0,2800.0,5,54250.0,12/1/2004,8/1/2012,PEUGEOT,2/3/2020,75,18015,7/1/2006,155000,4900,7500.0,1.0


In [3]:
sae.shape

(192880, 24)

### 1. Descriptive stats

In order to get a basic understanding of the dataset, I explore the null values & data type in each column:

In [4]:
sae.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192880 entries, 0 to 192879
Data columns (total 24 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   VehicleMarketCode          192880 non-null  object 
 1   VehicleNationalCode        192880 non-null  int64  
 2   VehicleSegmentationGlobal  192880 non-null  object 
 3   VehicleModelIntlName       192880 non-null  object 
 4   Bodytype_Group             192880 non-null  object 
 5   Fuel_Group                 192880 non-null  object 
 6   Transmission_Group         192880 non-null  object 
 7   Drivetype_Group            192880 non-null  object 
 8   VehicleKW                  192880 non-null  int64  
 9   VehicleDoors               192859 non-null  float64
 10  VehicleWheelBase           192834 non-null  float64
 11  VehicleSeats               192880 non-null  int64  
 12  VehicleGrossNewPrice       192880 non-null  float64
 13  VehicleProductionStart     19

In [5]:
null_cols=sae.isna().sum()
null_cols[null_cols>0]

VehicleDoors            21
VehicleWheelBase        46
oricpric               196
PriceChangeCount    119008
dtype: int64

In [6]:
null_cols=sae.isna().mean()*100
null_cols[null_cols>0]

VehicleDoors         0.010888
VehicleWheelBase     0.023849
oricpric             0.101618
PriceChangeCount    61.700539
dtype: float64

There are no relevant null values. Only the PriceChangeCount column has a high percentage 61.7% of null values. In normal circunstances I would delete the column but, in this case I will simply ignore it

##### Taking a look at the data type, it would be advisable to make some changes in order to perform possible calculations with them
* Natcode is the ID of AVG so, it should be a string instead of Integer
* Fields related to date are strings instead of datetime. I will set up as dates

In [7]:
sae['VehicleNationalCode']=sae['VehicleNationalCode'].astype('str')
sae['VehicleGrossNewPrice']=sae['VehicleGrossNewPrice'].astype('int')
sae['registration_date']=pd.to_datetime(sae['registration_date'], format='%m/%d/%Y')
sae['SAEReporteddate']=pd.to_datetime(sae['SAEReporteddate'], format='%m/%d/%Y')
sae['VehicleProductionStart']=pd.to_datetime(sae['VehicleProductionStart'], format='%m/%d/%Y')
sae['VehicleProductionEnd']=pd.to_datetime(sae['VehicleProductionEnd'], format='%m/%d/%Y', errors='coerce')

In [8]:
sae.dtypes

VehicleMarketCode                    object
VehicleNationalCode                  object
VehicleSegmentationGlobal            object
VehicleModelIntlName                 object
Bodytype_Group                       object
Fuel_Group                           object
Transmission_Group                   object
Drivetype_Group                      object
VehicleKW                             int64
VehicleDoors                        float64
VehicleWheelBase                    float64
VehicleSeats                          int64
VehicleGrossNewPrice                  int64
VehicleProductionStart       datetime64[ns]
VehicleProductionEnd         datetime64[ns]
VehicleMakeIntlName                  object
SAEReporteddate              datetime64[ns]
StockDays                             int64
PostCode                             object
registration_date            datetime64[ns]
Mileage                               int64
Price                                 int64
oricpric                        

### 2. Exploring the data in detail

#### 2.1 Categorical data 
I'm going to review the most relevant categorical columns from my point of view

In [9]:
sae.select_dtypes(exclude=['int64','float64'])

,VehicleMarketCode,VehicleNationalCode,VehicleSegmentationGlobal,VehicleModelIntlName,Bodytype_Group,Fuel_Group,Transmission_Group,Drivetype_Group,VehicleProductionStart,VehicleProductionEnd,VehicleMakeIntlName,SAEReporteddate,PostCode,registration_date
0,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,2004-07-01,2011-12-01,OPEL / VAUXHALL,2020-02-03,28983,2008-07-01
1,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,2004-07-01,2011-12-01,OPEL / VAUXHALL,2020-02-03,8940,2010-06-15
2,ES,29263,Small LCV,Combo,Panel van,Diesel,Manual,2WD,2004-07-01,2011-12-01,OPEL / VAUXHALL,2020-02-03,8840,2005-07-01
3,ES,30020,E,607,Sedan,Diesel,Automated,2WD,2004-12-01,2012-08-01,PEUGEOT,2022-10-12,27002,2009-10-15
4,ES,30020,E,607,Sedan,Diesel,Automated,2WD,2004-12-01,2012-08-01,PEUGEOT,2020-02-03,18015,2006-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192875,PT,934825,B,Micra,Hatchback,Petrol,Manual,2WD,2022-02-01,NaT,NISSAN,2022-10-12,1000,2022-05-15
192876,PT,934825,B,Micra,Hatchback,Petrol,Manual,2WD,2022-02-01,NaT,NISSAN,2022-10-12,3515,2022-04-15
192877,PT,934825,B,Micra,Hatchback,Petrol,Manual,2WD,2022-02-01,NaT,NISSAN,2022-10-12,8200,2022-03-15
192878,PT,934825,B,Micra,Hatchback,Petrol,Manual,2WD,2022-02-01,NaT,NISSAN,2022-10-12,2830,2022-05-15


##### a few questions to learn more about the data
How many markets/records appear in the dataset?

In [10]:
sae.VehicleMarketCode.value_counts()

ES    176407
PT     16473
Name: VehicleMarketCode, dtype: int64

What are manufacturers that contain more records?

In [11]:
sae.VehicleMakeIntlName.value_counts()

RENAULT       23370
PEUGEOT       17070
SEAT          15328
BMW           14385
FORD (EUR)    14078
              ...  
PIAGGIO           2
MPM MOTORS        1
BORGWARD          1
MORGAN            1
SAAB              1
Name: VehicleMakeIntlName, Length: 61, dtype: int64

What is the most repeated Natcode?

In [12]:
sae.VehicleNationalCode.value_counts()

74488     852
83098     652
80819     569
68495     567
56630     541
         ... 
56877       1
56901       1
56903       1
83619       1
934964      1
Name: VehicleNationalCode, Length: 15867, dtype: int64

The most repeated vehicle is the next:

In [13]:
sae[sae.VehicleNationalCode=='74488'].head()

,VehicleMarketCode,VehicleNationalCode,VehicleSegmentationGlobal,VehicleModelIntlName,Bodytype_Group,Fuel_Group,Transmission_Group,Drivetype_Group,VehicleKW,VehicleDoors,VehicleWheelBase,VehicleSeats,VehicleGrossNewPrice,VehicleProductionStart,VehicleProductionEnd,VehicleMakeIntlName,SAEReporteddate,StockDays,PostCode,registration_date,Mileage,Price,oricpric,PriceChangeCount
116852,ES,74488,B SUV,C-HR,Coupe SUV,Hybrid,Automated,2WD,90,5.0,2640.0,5,28990,2016-12-01,2019-10-01,TOYOTA,2020-02-03,29,29004,2018-07-01,38600,21990,21990.0,NaN
116853,ES,74488,B SUV,C-HR,Coupe SUV,Hybrid,Automated,2WD,90,5.0,2640.0,5,28990,2016-12-01,2019-10-01,TOYOTA,2020-02-03,17,25194,2019-07-01,3720,22200,22200.0,NaN
116854,ES,74488,B SUV,C-HR,Coupe SUV,Hybrid,Automated,2WD,90,5.0,2640.0,5,28990,2016-12-01,2019-10-01,TOYOTA,2022-10-12,5,46014,2019-09-15,47467,25750,25750.0,NaN
116855,ES,74488,B SUV,C-HR,Coupe SUV,Hybrid,Automated,2WD,90,5.0,2640.0,5,28990,2016-12-01,2019-10-01,TOYOTA,2022-10-12,2,33600,2018-07-01,36000,25900,25900.0,NaN
116856,ES,74488,B SUV,C-HR,Coupe SUV,Hybrid,Automated,2WD,90,5.0,2640.0,5,28990,2016-12-01,2019-10-01,TOYOTA,2020-02-03,16,46015,2019-07-01,11893,23900,23900.0,NaN


##### About date columns

As Carsten told me about Dataset, SAEReportedDate was the date the snippet was taken from. I'd like to check how many reports are on this dataset

In [14]:
sae.SAEReporteddate.value_counts()

2020-02-03    118597
2022-10-12     74283
Name: SAEReporteddate, dtype: int64

There are two reports in this dataset. Perhaps it is interesting to take it into account separately, since there have been external circumstances that have been able to alter the values (e.g. the global pandemic)

In [15]:
sae2020=sae[sae.SAEReporteddate=='2020-02-03']
sae2022=sae[sae.SAEReporteddate=='2022-10-12']

It may be interesting to create some additional columns in the dataset:
* New column with the **"age of the vehicle"** according to its **registration_date** until the respective **SAEReportedDate**
* New column with the **"loss of value"** of the vehicle according to **VehicleGrossNewPrice** and his current **Price**

In [16]:
sae['AgeOfVeh']=(sae['SAEReporteddate'] - sae['registration_date'])/np.timedelta64(1,'D')
sae['LossVal']=(sae['VehicleGrossNewPrice']-sae['Price'])

Based on that, also it is interesting to know the next variables:
* The relation between **"loss of value"** according to the **Mileage** in order to know the devaluation per distance
* The relation between **"loss of value"** according to the **AgeOfVeh** in order to know the devaluation per time
* The relation between **"Mileage"** according to the **AgeOfVeh** in order to know the distance per time

In [17]:
sae['DevPerKm']=(sae['LossVal']/sae['Mileage'])
sae['DevPerAge']=(sae['LossVal']/sae['AgeOfVeh'])
sae['KmPerAge']=(sae['Mileage']/sae['AgeOfVeh'])

In [18]:
sae.pivot_table(index=['VehicleMarketCode'], columns=['SAEReporteddate'], aggfunc='mean')

AgeOfVeh               DevPerAge              DevPerKm  \
SAEReporteddate    2020-02-03   2022-10-12 2020-02-03 2022-10-12 2020-02-03   
VehicleMarketCode                                                             
ES                 1412.12425  1897.737191  13.419215   6.645931   1.373761   
PT                 1431.61314  1946.491854   8.795999   4.610226   0.314525   

                               KmPerAge                  LossVal  \
SAEReporteddate   2022-10-12 2020-02-03 2022-10-12    2020-02-03   
VehicleMarketCode                                                  
ES                  0.408987  49.388000  47.210113  12938.089638   
PT                  0.141568  59.018516  53.215307  10093.014350   

                                      Mileage                        Price  \
SAEReporteddate      2022-10-12    2020-02-03     2022-10-12    2020-02-03   
VehicleMarketCode                                                            
ES                 10699.460478  68701.517088   86884.679629  16832.019462   
PT                  8610.318741  84081.331357  101563.574058  15039.743952   

                                PriceChangeCount             StockDays  \
SAEReporteddate      2022-10-12       2020-02-03 2022-10-12 2020-02-03   
VehicleMarketCode                                                        
ES                 21377.179025         1.938592   1.850572  78.239822   
PT                 18750.737097         1.812662   1.616865  86.357922   

                             VehicleDoors            VehicleGrossNewPrice  \
SAEReporteddate   2022-10-12   2020-02-03 2022-10-12           2020-02-03   
VehicleMarketCode                                                           
ES                 72.838018     4.685257   4.717294         29770.109100   
PT                 91.244373     4.811670   4.821415         25132.758302   

                                 VehicleKW             VehicleSeats  \
SAEReporteddate      2022-10-12 2020-02-03  2022-10-12   2020-02-03   
VehicleMarketCode                                                     
ES                 32076.639504  98.361067  103.565774     4.911097   
PT                 27361.055839  77.319616   85.251834     4.918287   

                             VehicleWheelBase                   oricpric  \
SAEReporteddate   2022-10-12       2020-02-03   2022-10-12    2020-02-03   
VehicleMarketCode                                                          
ES                   4.88933      2673.098723  2675.497191  17186.069278   
PT                   4.93471      2624.070090  2637.866683  15339.980313   

                                 
SAEReporteddate      2022-10-12  
VehicleMarketCode                
ES                 21664.264558  
PT                 18777.759483

#### 2.2 Numerical data
In order to look at multiple descriptive statistics at once, I use the describe function. This will show the count, mean, standard deviation, minimum, 25th percentile, median (50th percentile), 75th percentile, and the maximum.

In [19]:
sae.describe().T

,count,mean,std,min,25%,50%,75%,max
VehicleKW,192880.0,98.682186,47.152377,4.000000,73.000000,88.000000,110.000000,5.860000e+02
VehicleDoors,192859.0,4.707465,0.696083,2.000000,5.000000,5.000000,5.000000,6.000000e+00
VehicleWheelBase,192834.0,2670.309411,180.183084,1686.000000,2580.000000,2648.000000,2728.000000,4.522000e+03
VehicleSeats,192880.0,4.904920,0.725362,1.000000,5.000000,5.000000,5.000000,9.000000e+00
VehicleGrossNewPrice,192880.0,30259.096283,18962.733322,7140.000000,20450.000000,25850.000000,33390.000000,4.072500e+05
StockDays,192880.0,77.281688,75.502168,0.000000,19.000000,53.000000,110.000000,4.010000e+02
Mileage,192880.0,76988.615009,57069.621542,1.000000,27780.750000,69000.000000,113727.750000,3.500000e+05
Price,192880.0,18394.630418,13789.559508,1390.000000,10990.000000,15400.000000,21028.000000,4.159000e+05
oricpric,192684.0,18706.690836,16686.700415,-1.000000,11100.000000,15500.000000,21500.000000,2.390000e+06
PriceChangeCount,73872.0,1.889173,1.773183,1.000000,1.000000,1.000000,2.000000,9.500000e+01


The two main measures of central tendency are the mean and the median (50th percentile). Those variables that are close in mean and median value, means the data is symmetrically distributed around the mean.

On most important fields considered by me for the dataset, I'm seeing the mean is greater than the median, so data is right skewed. This means there are a few outliers that influence the mean.

#### Cleaning some typing errors

Quick look to oricpric column where the max value is highest than VehicleGrossNewPrice

In [20]:
sae[(sae.oricpric==2390000)]

,VehicleMarketCode,VehicleNationalCode,VehicleSegmentationGlobal,VehicleModelIntlName,Bodytype_Group,Fuel_Group,Transmission_Group,Drivetype_Group,VehicleKW,VehicleDoors,VehicleWheelBase,VehicleSeats,VehicleGrossNewPrice,VehicleProductionStart,VehicleProductionEnd,VehicleMakeIntlName,SAEReporteddate,StockDays,PostCode,registration_date,Mileage,Price,oricpric,PriceChangeCount,AgeOfVeh,LossVal,DevPerKm,DevPerAge,KmPerAge
165374,ES,85177,C SUV,Sportage,Closed SUV,Diesel,Manual,2WD,85,5.0,2670.0,5,32200,2019-05-01,2020-10-01,KIA,2022-10-12,210,8201,2020-01-15,22500,26900,2390000.0,3.0,1001.0,5300,0.235556,5.294705,22.477522


##### Could be some typing errors in oricpric column that could be fixed before analysis. Lets take a look from highest to lowest sort values

In [21]:
sae.sort_values(by=['oricpric'], ascending=False)

,VehicleMarketCode,VehicleNationalCode,VehicleSegmentationGlobal,VehicleModelIntlName,Bodytype_Group,Fuel_Group,Transmission_Group,Drivetype_Group,VehicleKW,VehicleDoors,VehicleWheelBase,VehicleSeats,VehicleGrossNewPrice,VehicleProductionStart,VehicleProductionEnd,VehicleMakeIntlName,SAEReporteddate,StockDays,PostCode,registration_date,Mileage,Price,oricpric,PriceChangeCount,AgeOfVeh,LossVal,DevPerKm,DevPerAge,KmPerAge
165374,ES,85177,C SUV,Sportage,Closed SUV,Diesel,Manual,2WD,85,5.0,2670.0,5,32200,2019-05-01,2020-10-01,KIA,2022-10-12,210,8201,2020-01-15,22500,26900,2390000.0,3.0,1001.0,5300,0.235556,5.294705,22.477522
104344,ES,72203,Large LCV,Boxer 35,Panel van,Diesel,Manual,2WD,96,4.0,4035.0,3,35950,2016-05-01,2019-05-01,PEUGEOT,2022-10-12,5,46714,2019-07-01,75969,23499,2350000.0,1.0,1199.0,12451,0.163896,10.384487,63.360300
170239,ES,89414,B,Clio,Hatchback,Hybrid,Automated,2WD,103,5.0,2583.0,5,23460,2020-06-01,2022-01-01,RENAULT,2022-10-12,227,29600,2021-07-01,10400,20400,2070000.0,3.0,468.0,3060,0.294231,6.538462,22.222222
89277,ES,69004,Sport,Aventador,Cabrio,Petrol,Automated,4WD,515,2.0,2700.0,2,407250,2015-01-01,2019-02-01,LAMBORGHINI,2020-02-03,56,28029,2018-08-15,1470,415900,415900.0,NaN,537.0,-8650,-5.884354,-16.108007,2.737430
89274,ES,69004,Sport,Aventador,Cabrio,Petrol,Automated,4WD,515,2.0,2700.0,2,407250,2015-01-01,2019-02-01,LAMBORGHINI,2020-02-03,261,28029,2017-04-15,3890,399900,399900.0,NaN,1024.0,7350,1.889460,7.177734,3.798828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174971,ES,93204,C SUV,Formentor,Closed SUV,Plug-in Hybrid,Automated,2WD,180,5.0,2680.0,5,48840,2021-03-01,NaT,CUPRA,2022-10-12,148,8023,2021-02-15,21453,44900,NaN,6.0,604.0,3940,0.183657,6.523179,35.518212
175190,ES,93519,C,Megane IV,Hatchback,Plug-in Hybrid,Automated,2WD,117,5.0,2670.0,5,41425,2021-05-01,2022-04-01,RENAULT,2022-10-12,124,39011,2021-09-15,17796,29990,NaN,4.0,392.0,11435,0.642560,29.170918,45.397959
175252,ES,93728,B,Ibiza,Hatchback,CNG,Manual,2WD,66,5.0,2564.0,5,21960,2021-06-01,2022-06-01,SEAT,2022-10-12,129,8720,2021-07-15,15741,16400,NaN,2.0,454.0,5560,0.353218,12.246696,34.671806
175747,ES,93917,C,308,Hatchback,Diesel,Automated,2WD,96,5.0,2675.0,5,33680,2021-06-01,2022-08-01,PEUGEOT,2022-10-12,131,28805,2021-06-15,25786,22490,NaN,3.0,484.0,11190,0.433956,23.119835,53.276860


In [22]:
sae['oricpric']=sae['oricpric'].replace(to_replace=2390000, value=23900)
sae['oricpric']=sae['oricpric'].replace(to_replace=2350000, value=23500)
sae['oricpric']=sae['oricpric'].replace(to_replace=2070000, value=20700)

Checking the "oricpric" column in ascending order, it's not possible to fix more data in this approximation.

In [23]:
sae.sort_values(by=['oricpric'], ascending=True)

,VehicleMarketCode,VehicleNationalCode,VehicleSegmentationGlobal,VehicleModelIntlName,Bodytype_Group,Fuel_Group,Transmission_Group,Drivetype_Group,VehicleKW,VehicleDoors,VehicleWheelBase,VehicleSeats,VehicleGrossNewPrice,VehicleProductionStart,VehicleProductionEnd,VehicleMakeIntlName,SAEReporteddate,StockDays,PostCode,registration_date,Mileage,Price,oricpric,PriceChangeCount,AgeOfVeh,LossVal,DevPerKm,DevPerAge,KmPerAge
177519,PT,56530,A,i10,Hatchback,Petrol,Manual,2WD,62,5.0,2380.0,5,11200,2011-03-01,2012-11-01,HYUNDAI,2022-10-12,12,2705-906,2012-01-15,93576,7490,-1.0,1.0,3923.0,3710,0.039647,0.945705,23.853174
181620,PT,72456,B SUV,C4 Cactus,Closed SUV,Diesel,Manual,2WD,73,5.0,2595.0,5,24360,2014-11-01,2018-05-01,CITROEN,2022-10-12,159,3780-621,2017-05-15,64012,16900,-1.0,1.0,1976.0,7460,0.116541,3.775304,32.394737
176873,PT,53118,C,Giulietta,Hatchback,Diesel,Manual,2WD,77,5.0,2634.0,5,25600,2010-04-01,2013-03-01,ALFA ROMEO,2022-10-12,153,3780-621,2011-04-15,67021,12850,-1.0,1.0,4198.0,12750,0.190239,3.037161,15.964983
179093,PT,64600,C,Leon,Hatchback,Diesel,Manual,2WD,110,5.0,2634.0,5,32666,2013-01-01,2015-06-01,SEAT,2022-10-12,221,3780-621,2013-11-15,128275,16450,-1.0,1.0,3253.0,16216,0.126416,4.984937,39.432831
180310,PT,68121,C,Megane III Grandtour,Stationwagon,Diesel,Manual,2WD,81,5.0,2703.0,5,28180,2013-10-01,2015-05-01,RENAULT,2022-10-12,202,4785-625,2014-11-15,165326,11900,0.0,1.0,2888.0,16280,0.098472,5.637119,57.245845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174971,ES,93204,C SUV,Formentor,Closed SUV,Plug-in Hybrid,Automated,2WD,180,5.0,2680.0,5,48840,2021-03-01,NaT,CUPRA,2022-10-12,148,8023,2021-02-15,21453,44900,NaN,6.0,604.0,3940,0.183657,6.523179,35.518212
175190,ES,93519,C,Megane IV,Hatchback,Plug-in Hybrid,Automated,2WD,117,5.0,2670.0,5,41425,2021-05-01,2022-04-01,RENAULT,2022-10-12,124,39011,2021-09-15,17796,29990,NaN,4.0,392.0,11435,0.642560,29.170918,45.397959
175252,ES,93728,B,Ibiza,Hatchback,CNG,Manual,2WD,66,5.0,2564.0,5,21960,2021-06-01,2022-06-01,SEAT,2022-10-12,129,8720,2021-07-15,15741,16400,NaN,2.0,454.0,5560,0.353218,12.246696,34.671806
175747,ES,93917,C,308,Hatchback,Diesel,Automated,2WD,96,5.0,2675.0,5,33680,2021-06-01,2022-08-01,PEUGEOT,2022-10-12,131,28805,2021-06-15,25786,22490,NaN,3.0,484.0,11190,0.433956,23.119835,53.276860


#### Save the dataset updated

In [24]:
sae.to_csv('../data/sae_dataset_updated.csv', index=False)